In [2]:
%load_ext autoreload
%autoreload 2

The central concept of Genki Signals is the data buffer:

In [2]:
import os 
os.chdir('/Users/egill/dev/genki/genki-signals')

from genki_signals.buffers import DataBuffer

buffer = DataBuffer(maxlen=400)

The `max_size` here is optional, but it makes the buffer circular, like a deque or queue, we can append to the end of it and if we append more than `max_size` elements, the oldest ones will automatically be deleted.

Apart from that, a `DataBuffer` behaves similarly to a pandas `DataFrame`, except that series can be multi-dimensional. A `DataFrame` only maps names to `pandas` `Series`, which are similar to 1D `numpy` arrays. A `DataBuffer`, on the other hand, can map names to arrays of arbitrary dimension, whwere the first is assumed to represent time. For example, if we have a signal from an accelerometer, it is natural to think of it as a 3D signal - an array of shape `(n, 3)` for `n` samples. With a `DataBuffer`, we can do this, referring to the entire signal as `acc` and don't have to have 3 different signals, e.t. `acc_x`, `acc_y`, and `acc_z`. The convenience of this will become clear later when we start defining operations on signals. 

A buffer without data is like a book without words, so let's generate some data! We'll make a simple sine wave generator to begin with:

In [3]:
from genki_signals.data_sources import Sampler

def make_sine_wave(freq, amplitude, phase):
    def f(t):
        return amplitude * np.sin(freq * 2 * np.pi * t + phase)
    return f

sine_wave = make_sine_wave(3, 5, 0)
sine_sampler = Sampler(sources={'sine': sine_wave}, sample_rate=100)

The `sine_wave` here is just a simple function of time, it can generate data at arbitrary frequency. To create a data generator, we need a `Sampler` - in this case we use the `rate` parameter to create one that samples at 100 samples per second. The sampler can combine multiple data generators but requires a name to make sense of the resulting data points. 

The unit of the `t` parameter that gets passed to the sine function is seconds, which means the sine wave has a frequency of 3 cycles per second - note that time here is not some abstract x-axis but actually real time. Once this data source gets going it will generate a sine wave with frequency 3 Hz, sampled at 100 Hz. We can now generate data in a separate thread, leaving the main thread unblocked:

In [4]:
import time
from threading import Thread

from genki_signals.data_sources import SineWave

buffer = DataBuffer(maxlen=400)
sine_wave = SineWave(frequency=3, amplitude=5, phase=0)
source = Sampler(sources={'sine': sine_wave}, sample_rate=100)

def run_source():
    source.start()
    while True:
        buffer.extend(source.read())
        time.sleep(1 / 25)
        if not source.is_active:
            return
        
t = Thread(target=run_source)
t.start()

Here we import the `SineWave` class from genki signals. This works exactly the same as using the function we defined previously, but the parameters (frequency, amplitude, and phase) are stored as instance variables, which means we can change them post-hoc and the generated data will update in real time.

As soon as the `start` method of the data source is called, it starts generating data on a sampling rate of 100 Hz. This happens in a separate thread, so we are actually spawning two threads here: one to generate the data and another one to regularly poll the source for new data and put it in a buffer.

Calling `source.read()` returns all data that the source has generated since the last call to `read`. In this case we are calling `read` 25 times a second, but the data source is generating data at 100 Hz, so we should expect `read` to return 4 data points on average. 

Effectively, there are three frequencies at play here:
* The sine wave frequency (3 Hz)
* The sample rate (100 Hz)
* The _update frequency_ (25 Hz)

The distinction between sampling rate and update frequency is very useful. The sample rate is often determined by the domain in question (e.g. a microphone samples at 44100 Hz) whereas the update frequency depends on what you are doing with the data. If we are recording data and flushing the data to disk occasionally, a very low update frequency makes sense. If we are plotting data on a real time dashboard, we  want a higher update frequency to make the animation smooth.

Speaking of plots, a key feature of the data buffer is visualisation of the data as it is generated:

In [5]:
buffer.keys()

dict_keys(['timestamp', 'sine'])

In [6]:
buffer.plot(key='sine')

Figure(axes=[Axis(label='t', scale=LinearScale()), Axis(label='sine', orientation='vertical', scale=LinearScal…

Because we are using the class `SineWave`, we can actually edit the wave parameters with everything running:

In [7]:
from ipywidgets import interact, IntSlider, FloatSlider

@interact(amplitude=FloatSlider(min=1, max=5, step=0.5, value=5), frequency=IntSlider(min=1, max=7, value=3))
def set_wave_params(amplitude, frequency):
    sine_wave.amplitude = amplitude
    sine_wave.frequency = frequency

interactive(children=(FloatSlider(value=5.0, description='amplitude', max=5.0, min=1.0, step=0.5), IntSlider(v…

In [8]:
source.stop()

We can also use user input as a data source, for example through the mouse position, keys on a keyboard, microphone, or a Genki Wave smart ring:

In [9]:
from genki_signals.data_sources import MicDataSource, WaveDataSource, MouseDataSource, KeyboardDataSource

mouse_source = Sampler({'mouse_position': MouseDataSource()}, sample_rate=100)
keyboard_source = Sampler({'keyboard': KeyboardDataSource(keys=["enter", "shift"])}, sample_rate=100)

wave_source = WaveDataSource(ble_address='5C72E785-DAF5-1E32-599D-EBF56B8ECD5B')
mic_source = MicDataSource()

The `MouseDataSource` reads the position of the mouse, the `KeyboardDataSource` detects whether the user is pressing the specified keys, `WaveDataSource` streams sensor data from a Wave ring, and the `MicDataSource` records audio from the microphone.

There is a difference between `MouseDataSource` and `KeyboardDataSource` on the one hand and `WaveDataSource` and `MicDataSource` on the other. The former two need to be wrapped in a `Sampler` whereas the latter do not. The reason for this is that the mouse position and keyboard pressing information (like the sine wave) can be accessed/computed at any time whereas the Wave ring and the microphone sample data on their own frequencies which we have no control over (short of resampling the data). In a way, they act as a data source and a sampler combined. 

In [10]:
buffer = DataBuffer(maxlen=400)
source = wave_source

def run_source():
    source.start()
    while True:
        buffer.extend(source.read())
        time.sleep(1 / 25)
        if not source.is_active:
            return
        
t = Thread(target=run_source)
t.start()

Connecting to wave at address 5C72E785-DAF5-1E32-599D-EBF56B8ECD5B
Connected to Wave


In [11]:
buffer.keys()

dict_keys(['gyro', 'acc', 'mag', 'raw_pose', 'current_pose', 'euler', 'linacc', 'peak', 'peak_norm_velocity', 'timestamp_us', 'grav', 'acc_glob', 'linacc_glob'])

Some of the signals from the Wave ring are defined as multi-dimensional, for example we can plot the 3D gyroscope signal as a "single thing":

In [12]:
buffer.plot(key='gyro')

Figure(axes=[Axis(label='t', scale=LinearScale()), Axis(label='gyro', orientation='vertical', scale=LinearScal…

In [13]:
source.stop()

Got a cancel message, exiting.


## Derived signals

The real power of Genki Signals comes from using derived signals:

In [29]:
import genki_signals.signals as s

derived_signals = [
    s.Sum('sine_0', 'sine_1', name='composite_sine'),
    s.Sum('composite_sine', 'noise', name='wave_with_noise'),
    s.FourierTransform('wave_with_noise', name='wave_spectrum', window_size=256, window_overlap=128),
    s.SampleRate('timestamp', name='sample_rate'),
    s.MovingAverage('sample_rate', length=10, name='smooth_sample_rate'),
]

What are `derived_signals`? It specifies a configuration of derived signals in the context of some data sources. Adding `sine_0` and `sine_1` is meaningless without knowing what `sine_0` and `sine_1` are.

What do we want from an object like `derived_signals`?

* It represents a DAG of time-series operations, each operation can take source signals or results of other operations as input. Names are important!
* These should work both offline and online (real time). They are _causal_ in other words.
* They should be deterministic - reproducible from the specification and input data only and can only depend on local state.
* Determinism means they are serializable to e.g. JSON - this can be passed from frontend to backend in a web app 
* We can create a torch module (and sklearn pipeline) from a given list of derived signals and input/output names - which is then serializable as ONNX or tf-lite
* This is tensor backend-agnostic. Each signal is capable of working with numpy arrays, torch tensors, etc.
* Each signal in the list is a function that can operate independently given some data.

We are still not computing any of the derived signals. If we want to use these signals, we need to manage all the names and call the signals with the right inputs at each step. This is exactly what a `System` does:

In [14]:
from genki_signals.system import System
from genki_signals.data_sources import RandomNoise

sine_1 = SineWave(frequency=3, amplitude=5,phase=0)
sine_2 = SineWave(frequency=7, amplitude=2, phase=0)
noise_source = RandomNoise()

data_source = Sampler({
    'sine_0': sine_1,
    'sine_1': sine_2,
    'noise': noise_source
}, sample_rate=100)

buffer = DataBuffer(maxlen=400)

system = System(
    data_source=data_source,
    derived_signals=derived_signals
)

In [15]:
import time
from threading import Thread

def run_system():
    system.start()
    while True:
        buffer.extend(system.read())
        time.sleep(1 / 25)
        if not system.is_active: 
            return
        
t = Thread(target=run_system)
t.start()

In [16]:
buffer.keys()

dict_keys(['timestamp', 'sine_0', 'sine_1', 'noise', 'composite_sine', 'wave_with_noise', 'wave_spectrum', 'sample_rate', 'smooth_sample_rate'])

In [17]:
buffer.plot('wave_with_noise')

Figure(axes=[Axis(label='t', scale=LinearScale()), Axis(label='wave_with_noise', orientation='vertical', scale…

In [18]:
from ipywidgets import interact, FloatSlider

@interact(noise=FloatSlider(min=0.1, max=3.0, step=0.1, value=1))
def set_noise(noise):
    noise_source.amplitude = noise

interactive(children=(FloatSlider(value=1.0, description='noise', max=3.0, min=0.1), Output()), _dom_classes=(…

Setting `plot_type` changes the type of plot created. Genki signals includes a few widgets for data visualisation. Here are a few examles:

In [20]:
buffer.plot(key='wave_spectrum', plot_type='spectrogram', sample_rate=100, window_size=256)

Figure(axes=[Axis(label='Hz', scale=LinearScale()), Axis(label='db', orientation='vertical', scale=LinearScale…

We see spikes at frequencies 3 and 7 Hz, as expected. Note we can still adjust the noise and the spikes will become more/less clear

In [21]:
system.stop()

We can add derived signals to a running system:

In [22]:
mic_source = MicDataSource()

buffer = DataBuffer(maxlen=400)


system = System(
    data_source=mic_source,
    derived_signals=[]
)

def run_system():
    system.start()
    while True:
        time.sleep(1 / 25)
        buffer.extend(system.read())
        if not system.is_active:
            return
        
t = Thread(target=run_system)
t.start()

In [23]:
buffer.keys()

dict_keys(['audio'])

In [24]:
system.add_derived_signal(s.FourierTransform('audio', 'audio_spectrum', window_size=1024, window_overlap=0, upsample=False))

In [25]:
buffer.keys()

dict_keys(['audio', 'audio_spectrum'])

In [26]:
buffer.plot(key='audio_spectrum', plot_type='spectrogram', sample_rate=mic_source.sample_rate, window_size=1024)

Figure(axes=[Axis(label='Hz', scale=LinearScale()), Axis(label='db', orientation='vertical', scale=LinearScale…

In [27]:
system.stop()

Note that since we pass `upsample=False` to the signal, the spectral signals is generated at a much lower frequency than the original (audio) one.

In [24]:
mouse_system = System(mouse_source)
buffer = DataBuffer(maxlen=400)

def run_system():
    mouse_system.start()
    while True:
        buffer.extend(mouse_system.read())
        time.sleep(1 / 50)
        if not mouse_system.is_active:
            return
        
t = Thread(target=run_system)
t.start()

In [25]:
buffer.plot(key='mouse_position', plot_type="trace2D")

Figure(axes=[Axis(label='x', scale=LinearScale()), Axis(label='y', orientation='vertical', scale=LinearScale()…

In [26]:
mouse_system.stop()

# A note on clocks

So far, some of the data sources we have seen have had to be wrappped in a `Sampler` to be used with the rest of Genki Signals, whereas some have not. We know the reason for this - external devices like a Wave ring have their own clock and sampling rate that we have no direct control over, packages just arrive via bluetooth when they do and synchronising clocks in a distributed system is virtually impossible.

What if we want to combine multiple sources? What if we want to use data from, say, a Wave ring and the mouse, or two separate Wave rings? A `Sampler` object can easily sample from multiple data generators at a time - the way a `Sampler` works is that it simply runs a busy thread that wakes up at the specified frequency and reads data from all its sources, since these are all a simple function of time, this works out pretty well.

But what if we want to mix and match sources that operate their own clock? The only sensible way to do this is to make one source the "leader" that works as the sampler in the example above, and make the others "followers" that work like the data generators. To make e.g. Wave a leader source with some followers, we would simply query all the followers for the current data point any time we receive a package from Wave. To make Wave act like a follower, we would need to run the receiver in a separate thread, and always keep the latest data point to answer queries from the leader. If the leader is operating at a lower sampling rate than Wave, some data points are simply ignored (implicit downsampling), whereas if the leader has a higher sampling rate, identical data points will be returned more than once (implicit upsampling). 

# Model inference

Perhaps the most useful class of signals implemented by the system is model inference signals, the simplest one of which is `Inference`:

In [32]:
from genki_signals.models.letter_detection_model import SimpleGruModel

example_rnn_model = SimpleGruModel.load_from_checkpoint("genki_signals/models/stc_detector_final-epoch=15-val_loss=0.53.ckpt")

derived = [
    s.Differentiate("mouse_position", sig_b="timestamp", name="mouse_velocity"),
    s.Inference(example_rnn_model, input_signal="mouse_velocity", stateful=True, name="rnn_model_inference")
]

mouse_source = Sampler({'mouse_position': MouseDataSource()}, sample_rate=100)
system = System(mouse_source, derived_signals=derived)
buffer = DataBuffer(maxlen=400)

def run_system():
    system.start()
    while True:
        buffer.extend(system.read())
        time.sleep(1 / 50)
        if not system.is_active:
            return
        
t = Thread(target=run_system)
t.start()

In [33]:
buffer.plot('rnn_model_inference', plot_type='histogram', class_names=['background', 'square', 'triangle', 'circle'])

Figure(axes=[Axis(scale=OrdinalScale()), Axis(label='Probability', orientation='vertical', scale=LinearScale()…

In [34]:
system.stop(); t.join()

Here we have a model that is a _recurrent neural network_. Those are particularly simple to run in real time systems since they make a prediction at each time step. At each step they output a prediction vector and a state vector, and require an input vector and the previious state vector as input. The `Inference` signal handles this for us. An even simpler model would be one that takes in an input and outputs a prediction at each step independently, in that case `Inference` would also work, but with `stateful=False`.

When you want to detect if/when the user drew a certain shape, outputting predictions at 100Hz is probably not exactly what you want. More likely is that you want to trigger some kind of event whenever it happened. You probably wouldn't want the event to be triggered repeatedly for what is actually the same movement

Some models would operate independently (i.e. without state) but on a windowed view of data at a time. A classic example of this are _Convolutional Neural Networks_:

In [30]:
derived = [
    s.Concatenate(['linacc', 'gyro'], name='model_input'),
    s.WindowedInference('genki_signals/models/model/model.onnx', 'model_input', 
                        name='swipe_inference', window_size=128, window_overlap=32, output_shape=(3,))
]

In [31]:
from genki_signals.system import System

#wave_source = WaveDataSource(ble_address='5C72E785-DAF5-1E32-599D-EBF56B8ECD5B')
system = System(wave_source, derived_signals=derived)
buffer = DataBuffer(maxlen=400)

def run_system():
    system.start()
    while True:
        buffer.extend(system.read())
        time.sleep(1 / 50)
        if not system.is_active:
            return
        
t = Thread(target=run_system)
t.start()

Exception in thread Thread-15:
Traceback (most recent call last):
  File "/Users/egill/opt/miniconda3/envs/genki/lib/python3.9/asyncio/selector_events.py", line 256, in _add_reader
    key = self._selector.get_key(fd)
  File "/Users/egill/opt/miniconda3/envs/genki/lib/python3.9/selectors.py", line 193, in get_key
    raise KeyError("{!r} is not registered".format(fileobj)) from None
KeyError: '94 is not registered'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/egill/opt/miniconda3/envs/genki/lib/python3.9/threading.py", line 973, in _bootstrap_inner
Exception in thread Thread-14:
Traceback (most recent call last):
  File "/Users/egill/opt/miniconda3/envs/genki/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/Users/egill/opt/miniconda3/envs/genki/lib/python3.9/threading.py", line 910, in run
    self.run()
  File "/Users/egill/opt/miniconda3/envs/genki/lib/python3.9/site-packages/

In [13]:
wave_source.start()

Exception in thread Thread-10:
Traceback (most recent call last):
  File "/Users/egill/opt/miniconda3/envs/genki/lib/python3.9/asyncio/selector_events.py", line 256, in _add_reader
    key = self._selector.get_key(fd)
  File "/Users/egill/opt/miniconda3/envs/genki/lib/python3.9/selectors.py", line 193, in get_key
    raise KeyError("{!r} is not registered".format(fileobj)) from None
KeyError: '77 is not registered'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/egill/opt/miniconda3/envs/genki/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/Users/egill/opt/miniconda3/envs/genki/lib/python3.9/site-packages/genki_wave/threading_runner.py", line 113, in run
    loop = get_or_create_event_loop()
  File "/Users/egill/opt/miniconda3/envs/genki/lib/python3.9/site-packages/genki_wave/utils.py", line 48, in get_or_create_event_loop
    asyncio.set_event_loop(asyncio.new_event_loop())
  Fil

In [7]:
from onnxruntime import InferenceSession

session = InferenceSession('genki_signals/models/model/model.onnx')

In [21]:
import numpy as np

output, output_extra = session.run(['output', 'output_extra'], {"input": np.ones((1, 128, 6)).astype(np.float32)})

In [22]:
output.shape, output_extra.shape

((1, 3, 16), (1, 6, 128))

In [38]:
s.WindowedInference

NameError: name 's' is not defined

In [3]:
# Swipe demo? Object detection post-processing?

In [ ]:
# Sklearn model demo?

Other things to add to demo:

* Demonstrate signal spec serialisation to e.g. ONNX
* Synthesise some complex data
* More signal processing magic - e.g. modulate a signal with noise and delay and demodulate, custom filter design with interactive parameter sliders


In [19]:
session.run?

Signature: session.run(output_names, input_feed, run_options=None)
Docstring:
Compute the predictions.

:param output_names: name of the outputs
:param input_feed: dictionary ``{ input_name: input_value }``
:param run_options: See :class:`onnxruntime.RunOptions`.

::

    sess.run([output_name], {input_name: x})
File:      ~/opt/miniconda3/envs/genki/lib/python3.9/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py
Type:      method

In [24]:
InferenceSession?

Init signature:
InferenceSession(
    path_or_bytes,
    sess_options=None,
    providers=None,
    provider_options=None,
    **kwargs,
)
Docstring:      This is the main class used to run a model.
Init docstring:
:param path_or_bytes: filename or serialized ONNX or ORT format model in a byte string
:param sess_options: session options
:param providers: Optional sequence of providers in order of decreasing
    precedence. Values can either be provider names or tuples of
    (provider name, options dict). If not provided, then all available
    providers are used with the default precedence.
:param provider_options: Optional sequence of options dicts corresponding
    to the providers listed in 'providers'.

The model type will be inferred unless explicitly set in the SessionOptions.
To explicitly set:

::

    so = onnxruntime.SessionOptions()
    # so.add_session_config_entry('session.load_model_format', 'ONNX') or
    so.add_session_config_entry('session.load_model_format', 'ORT')

